In [5]:
# 191805061-Salih Can Aydoğdu, 
# 201805051-Utku Enes Baki(team leader),
# 191805060-Semih Utku Polat,
# 201805044-Murat Tan

In [6]:
# configurations of PC

import socket
import platform
import re
import uuid
import psutil

# address informations
print('Address:')
print('Ip: ', socket.gethostbyname(socket.gethostname()))
print('Mac: ', ':'.join(re.findall('..', '%012x' % uuid.getnode())))

print()

# platform informations
print('Platform:')
print('Computer (Host) Name: ', platform.node())
print('OS: ' + platform.system())
print('Release: ' + platform.release())
print('Version: ' + platform.version())
print('Architecture: ' + platform.machine())
print('Processor: ' + platform.processor())

print()

# CPU informations
print('CPU:')
print('Physical Cores', psutil.cpu_count(logical = False)) # except logical cores
print('Total Cores:', psutil.cpu_count(logical = True))
print('Frequency:', psutil.cpu_freq().current / 1000, 'GHz') # megahertz to gigahertz

print()

# memory informations
memory = psutil.virtual_memory()
print('Memory:')
print('Total:', memory.total / 1024**3, 'GB') # bytes to gigabytes
print('Available:', memory.available / 1024**3, 'GB')
print('Used:', memory.used / 1024**3, 'GB')

print()

# disk informations
disk = psutil.disk_usage('/') # root directory (C:\)
print('Disk:')
print('Total:', disk.total / 1024**3, 'GB')
print('Used:', disk.used / 1024**3, 'GB')
print('Free:', disk.free / 1024**3, 'GB')

Address:
Ip:  192.168.1.39
Mac:  d8:cb:8a:f3:dd:59

Platform:
Computer (Host) Name:  DESKTOP-UD2G1JC
OS: Windows
Release: 10
Version: 10.0.19045
Architecture: AMD64
Processor: Intel64 Family 6 Model 94 Stepping 3, GenuineIntel

CPU:
Physical Cores 4
Total Cores: 4
Frequency: 2.301 GHz

Memory:
Total: 7.910999298095703 GB
Available: 1.3878593444824219 GB
Used: 6.523139953613281 GB

Disk:
Total: 311.01855087280273 GB
Used: 256.24313735961914 GB
Free: 54.775413513183594 GB


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, LongType

from pyspark.sql.functions import split, explode, sum, lit, col, when, asc, desc, avg, mean, count, isnan

# start spark session at single local node with one thread
spark = SparkSession.builder.master('local[1]').appName('191805060_201805051_191805061_201805044').config('spark.driver.extraJavaOptions', '-Xss3g').getOrCreate()

# enable eager evaluation of Spark SQL expressions in the REPL (Read-Eval-Print Loop)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

# path of datasets
path_to_movies_csv = 'C:/spark/final/Netflix_Dataset_Movie.csv'
path_to_ratings_csv = 'C:/spark/final/Netflix_Dataset_Rating.csv'

# read datasets first rows as column names (columns may nullable) and save to DataFrame(df) objects - CSV (Comma Separated Values)
movies = spark.read \
    .option('header', True) \
    .option('inferschema', True) \
    .schema(StructType([
        StructField('Movie_ID', LongType(), True),
        StructField('Year', LongType(), True),
        StructField('Name', StringType(), True)
    ])) \
    .csv(path_to_movies_csv)

ratings = spark.read \
    .option('header', True) \
    .option('inferschema', True) \
    .schema(StructType([
        StructField('User_ID', LongType(), True),
        StructField('Rating', DoubleType(), True),
        StructField('Movie_ID', LongType(), True)
    ])) \
    .csv(path_to_ratings_csv)

ModuleNotFoundError: No module named 'pyspark'

In [8]:
# "movies" dataset
# columns informations
movies.printSchema()

print()

# total row count
print('Total row count of "movies" data frame: ', movies.count())

print()

# first 20 rows
# movies.show(20, truncate = True) #- when not using jupyter notebook (more stable)
movies

NameError: name 'movies' is not defined

In [9]:
# "ratings" dataset
# columns informations
ratings.printSchema()

print()

# total row count
print('Total row count of "ratings" data frame: ', ratings.count())

print()

# first 20 rows
# ratings.show(20, truncate = True)
ratings

NameError: name 'ratings' is not defined

In [10]:
# join the ratings, users, and books data frames
# how = 'inner' returns only the rows that have matching values in all DataFrames being joined
users_who_rate = ratings.join(movies, 'Movie_ID', how = 'inner') \
                .select(col('Movie_ID'), col('Name'), col('Year'), col('User_ID'), col('Rating')) \
                .orderBy(col('User_ID').asc())

NameError: name 'ratings' is not defined

In [11]:
# columns informations
users_who_rate.printSchema()

print()

# users rates total row count
print('User rates total row count: ', users_who_rate.count())

print()

# users rates informations
users_who_rate.select('Rating').describe()

NameError: name 'users_who_rate' is not defined

In [12]:
# first 20 rows
# users_who_rate.show(20, truncate = True)
users_who_rate

NameError: name 'users_who_rate' is not defined

In [13]:
from pyspark.ml.feature import VectorAssembler, Normalizer, PCA, VectorSlicer, StandardScaler
from pyspark.ml.linalg import Vectors

# create user-item matrix
matrix = users_who_rate.groupBy("User_ID").pivot("Movie_ID").agg(avg("Rating")).fillna(0)

# create vector assembler
assembler = VectorAssembler(inputCols=matrix.columns[1:], outputCol="features")

# assemble the features into a vector column
data = assembler.transform(matrix)

# normalize the features
normalizer = Normalizer(inputCol="features", outputCol="normFeatures")
normalizedData = normalizer.transform(data)

# calculate pairwise cosine similarities
pca = PCA(k=10, inputCol="normFeatures", outputCol="pcaFeatures")
model = pca.fit(normalizedData)
transformedData = model.transform(normalizedData)
similarities = transformedData.select("User_ID", "pcaFeatures").rdd.mapValues(lambda v: Vectors.dense(v.toArray())).toDF(["User_ID", "features"]).alias("a").crossJoin(transformedData.alias("b")).where("a.User_ID < b.User_ID").selectExpr("a.User_ID as user1", "b.User_ID as user2", "pearson(features, b.pcaFeatures) as similarity")

# generate recommendations for a given user
userId = 1
userRatings = ratings.filter(col("User_ID") == userId).select("Movie_ID", "Rating").rdd.map(lambda r: (r[0], r[1])).collectAsMap()
slicer = VectorSlicer(inputCol="pcaFeatures", outputCol="features", indices=[i-1 for i in userRatings.keys()])
selectedData = slicer.transform(transformedData.filter(col("User_ID").isin(userRatings.keys())))
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaledData = scaler.fit(selectedData).transform(selectedData)
scaledData = scaledData.select("User_ID", "Movie_ID", "scaledFeatures", "Rating").rdd.map(lambda r: (r[0], r[1], Vectors.dense(r[2].toArray()), r[3])).toDF(["User_ID", "Movie_ID", "scaledFeatures", "Rating"])
predictions = model.transform(scaledData)
predictions = predictions.rdd.map(lambda r: (r[0], r[1], r[6])).filter(lambda r: r[0] == userId).sortBy(lambda r: -r[2]).take(10)

# print recommendations
print("Top 10 recommendations for User %d:" % userId)
for prediction in predictions:
    print("Movie %d: predicted rating = %f" % (prediction[1], prediction[2]))

ModuleNotFoundError: No module named 'pyspark'

In [14]:
users_who_rate.filter(users_who_rate.Movie_ID == 1000)

NameError: name 'users_who_rate' is not defined

In [15]:
from pyspark.sql.functions import col, avg
from pyspark.ml.feature import VectorAssembler, Normalizer, PCA, VectorSlicer, StandardScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.recommendation import ALS


# set spark.sql.pivotMaxValues parameter to a higher value
spark.conf.set("spark.sql.pivotMaxValues", "100000")


# create user-item matrix
matrix = users_who_rate.groupBy("User_ID").pivot("Movie_ID").agg(avg("Rating")).fillna(0)

# create vector assembler
assembler = VectorAssembler(inputCols=matrix.columns[1:], outputCol="features")

# assemble the features into a vector column
data = assembler.transform(matrix)

# normalize the features
normalizer = Normalizer(inputCol="features", outputCol="normFeatures")
normalizedData = normalizer.transform(data)

# calculate pairwise cosine similarities
pca = PCA(k=10, inputCol="normFeatures", outputCol="pcaFeatures")
model = pca.fit(normalizedData)
transformedData = model.transform(normalizedData)
similarities = transformedData.select("User_ID", "pcaFeatures").rdd.mapValues(lambda v: Vectors.dense(v.toArray())).toDF(["User_ID", "features"]).alias("a").crossJoin(transformedData.alias("b")).where("a.User_ID < b.User_ID").selectExpr("a.User_ID as user1", "b.User_ID as user2", "pearson(features, b.pcaFeatures) as similarity")

# generate recommendations for a given user
userId = 1
userRatings = ratings.filter(col("User_ID") == userId).select("Movie_ID", "Rating").rdd.map(lambda r: (r[0], r[1])).collectAsMap()
slicer = VectorSlicer(inputCol="pcaFeatures", outputCol="features", indices=[i-1 for i in userRatings.keys()])
selectedData = slicer.transform(transformedData.filter(col("User_ID").isin(userRatings.keys())))
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaledData = scaler.fit(selectedData).transform(selectedData)
scaledData = scaledData.select("User_ID", "Movie_ID", "scaledFeatures", "Rating").rdd.map(lambda r: (r[0], r[1], Vectors.dense(r[2].toArray()), r[3])).toDF(["User_ID", "Movie_ID", "scaledFeatures", "Rating"])
predictions = model.transform(scaledData)
predictions = predictions.rdd.map(lambda r: (r[0], r[1], r[6])).filter(lambda r: r[0] == userId).sortBy(lambda r: -r[2]).take(10)

# print recommendations
print("Top 10 recommendations for User %d:" % userId)
for prediction in predictions:
    print("Movie %d: predicted rating = %f" % (prediction[1], prediction[2]))

ModuleNotFoundError: No module named 'pyspark'

In [16]:
# visualization

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
movie_rating_counts = users_who_rate.rdd \
    .map(lambda x: (x[0], (x[4], 1))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

NameError: name 'users_who_rate' is not defined

In [ ]:
for movie in movie_rating_counts.collect():
    print(movie)

In [ ]:
# join the ratings, users, and books data frames
# how = 'inner' returns only the rows that have matching values in all DataFrames being joined
users_who_rate_for_prediction = ratings.join(movies, 'Movie_ID', how = 'inner') \
                .select(col('Movie_ID'), col('User_ID'), col('Rating')) \
                .orderBy(col('User_ID').asc())

In [ ]:
# columns informations
users_who_rate_for_prediction.printSchema()

print()

# users rates total row count
print('User rates total row count: ', users_who_rate_for_prediction.count())

print()

# users rates informations
users_who_rate_for_prediction.select('Rating').describe()

In [ ]:
# users_who_rate_for_prediction.show(20, truncate = True)
users_who_rate_for_prediction

In [ ]:
# is there a NaN or null value in user rating
users_who_rate_for_prediction.select([count(when(isnan('Rating') | col('Rating').isNull(), 'Rating'))])

In [ ]:
# is there a NaN or null value in book id
users_who_rate_for_prediction.select([count(when(isnan('Movie_ID') | col('Movie_ID').isNull(), 'Movie_ID'))])

In [ ]:
# is there a NaN or null value in user id
users_who_rate_for_prediction.select([count(when(isnan('User_ID') | col('User_ID').isNull(), 'User_ID'))])

In [ ]:
# recommendation engine

# use %70 of dataset for training, rest of them for testing
# randomly selected rows from users_who_rate_for_prediction with reference of seed
# data will split in the same way every time with same seed (use seed for consistent results)
train_data, test_data = users_who_rate_for_prediction.randomSplit([0.7, 0.3], seed = 5060)

In [ ]:
# train data total row count
print('Train data total row count: ', train_data.count())

print()

train_data = train_data.orderBy('User_ID', ascending = True)
train_data.show(200, truncate = True)

In [ ]:
# test data total row count
print('Test data total row count: ', test_data.count())

print()

test_data = test_data.orderBy('User_ID', ascending = True)
test_data.show(200, truncate = True)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
results = []

def predict_model(rank, maxIter, regParam) :
    
    # ALS
    als = ALS(userCol = 'User_ID', itemCol = 'Movie_ID', ratingCol = 'Rating', seed = 5060, implicitPrefs = False, nonnegative = True, coldStartStrategy = 'drop', rank = rank, maxIter = maxIter, regParam = regParam)
    model = als.fit(train_data)
    predictions = model.transform(test_data)
    predictions = predictions.withColumnRenamed('prediction', 'Prediction')

    # evaluate model

    # rmse (root mean squared error)
    rmse_evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'Rating', predictionCol = 'Prediction')
    rmse = rmse_evaluator.evaluate(predictions)

    # mse (mean squared error)
    mse_evaluator = RegressionEvaluator(metricName = 'mse', labelCol = 'Rating', predictionCol = 'Prediction')
    mse = mse_evaluator.evaluate(predictions)
    
    return {'predictions' : predictions, 'results': {'rank': rank, 'iteration': maxIter, 'lambda': regParam, 'RMSE': rmse, 'MSE': mse}}

In [ ]:
model_1_1 = predict_model(10, 10, 0.01)

print(model_1_1['results'])

print()

model_1_1['predictions'].show(500, truncate = True)

In [ ]:
model_1_2 = predict_model(10, 50, 0.01)


print(model_1_2['results'])

print()

model_1_2['predictions'].show(200, truncate = True)

In [ ]:
model_1_3 = predict_model(10, 200, 0.01)


print(model_1_3['results'])

print()

model_1_3['predictions'].show(200, truncate = True)

In [ ]:
model_2_1 = predict_model(50, 10, 0.01)


print(model_2_1['results'])

print()

model_2_1['predictions'].show(200, truncate = True)

In [ ]:
model_2_2 = predict_model(50, 50, 0.01)


print(model_2_2['results'])

print()

model_2_2['predictions'].show(200, truncate = True)

In [ ]:
model_2_3 = predict_model(50, 200, 0.01)


print(model_2_3['results'])

print()

model_2_3['predictions'].show(200, truncate = True)

In [ ]:
model_3_1 = predict_model(200, 10, 0.01)


print(model_3_1['results'])

print()

model_3_1['predictions'].show(200, truncate = True)

In [ ]:
model_3_2 = predict_model(200, 50, 0.01)


print(model_3_2['results'])

print()

model_3_2['predictions'].show(200, truncate = True)

In [ ]:
model_3_3 = predict_model(200, 200, 0.01)


print(model_3_3['results'])

print()

model_3_3['predictions'].show(200, truncate = True)

In [ ]:
model_1_1_1 = predict_model(10, 10, 0.1)

print(model_1_1_1['results'])

print()

model_1_1_1['predictions'].show(500, truncate = True)

In [ ]:
model_1_2_2 = predict_model(10, 50, 0.1)


print(model_1_2_2['results'])

print()

model_1_2_2['predictions'].show(200, truncate = True)

In [ ]:
model_1_3_3 = predict_model(10, 200, 0.1)


print(model_1_3_3['results'])

print()

model_1_3_3['predictions'].show(200, truncate = True)

In [ ]:
model_2_1_1 = predict_model(50, 10, 0.1)


print(model_2_1_1['results'])

print()

model_2_1_1['predictions'].show(200, truncate = True)

In [ ]:
model_2_2_2 = predict_model(50, 50, 0.1)


print(model_2_2_2['results'])

print()

model_2_2_2['predictions'].show(200, truncate = True)

In [ ]:
model_2_3_3 = predict_model(50, 200, 0.1)


print(model_2_3_3['results'])

print()

model_2_3_3['predictions'].show(200, truncate = True)

In [ ]:
model_3_1_1 = predict_model(200, 10, 0.1)


print(model_3_1_1['results'])

print()

model_3_1_1['predictions'].show(200, truncate = True)

In [ ]:
model_3_2_2 = predict_model(200, 50, 0.1)


print(model_3_2_2['results'])

print()

model_3_2_2['predictions'].show(200, truncate = True)

In [ ]:
model_3_3_3 = predict_model(200, 200, 0.1)


print(model_3_3_3['results'])

print()

model_3_3_3['predictions'].show(200, truncate = True)

In [ ]:
results.append(model_1_1['results'])
results.append(model_1_2['results'])
results.append(model_1_3['results'])
results.append(model_2_1['results'])
results.append(model_2_2['results'])
results.append(model_2_3['results'])
results.append(model_3_1['results'])
results.append(model_3_2['results'])
results.append(model_3_3['results'])

results.append(model_1_1_1['results'])
results.append(model_1_2_2['results'])
results.append(model_1_3_3['results'])
results.append(model_2_1_1['results'])
results.append(model_2_2_2['results'])
results.append(model_2_3_3['results'])
results.append(model_3_1_1['results'])
results.append(model_3_2_2['results'])
results.append(model_3_3_3['results'])

results_data_frame = spark.createDataFrame(results)
# results_data_frame.show(20, truncate = True)
results_data_frame

In [ ]:
# best model for RMSE
best_model_of_RMSE = sorted(results, key=lambda k: k['RMSE'])
print('Best model for RMSE:', best_model_of_RMSE[0])

In [ ]:
# best model for MSE
best_model_of_MSE = sorted(results, key=lambda k: k['MSE'])
print('Best model for MSE:', best_model_of_MSE[0])

In [20]:
#cosine simliarity model to product
import numpy as np
from numpy.linalg import norm
#test_data
#results_data_frame
test_cosine= test_data.to_numpy()
results_cosine=  results_data_frame.to_numpy()
print("A:\n", test_cosine)
print("B:\n", results_cosine)
 
# compute cosine similarity
cosine = np.sum(test_cosine*results_cosine, axis=1)/(norm(test_cosine, axis=1)*norm(results_cosine, axis=1))
 
print("Cosine Similarity:\n", cosine)

NameError: name 'test_data' is not defined

In [ ]:
# filter specific row
# users.filter(users.user_id == 278858)

#users = users.dropna()
#df.dropna(inplace=True)
#users


#predictions.select([count(when(isnan('rating') | col('rating').isNull(), 'rating'))])



# users_who_rate_for_prediction = users_who_rate_for_prediction.filter(col('user_rating') != 0)